Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.60.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

myfreeworkspace
myfreeresource
eastus2
18232d0b-4c73-4051-8694-16fb46b5a555


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'myfirstauroml'
project_folder = './pipeline-bankmarketing-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
myfirstauroml,myfreeworkspace,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "mycompute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [6]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bankmarketing"
description_text = "BankMarketing DataSet for Udacity Course 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [7]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 1,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",  
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [10]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [11]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [12]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [13]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [6b823867][5e3c4832-f301-42f6-b92d-f88f855ec85a], (This step will run and generate new outputs)
Submitted PipelineRun 0ba27ce5-d963-45eb-b3b6-56afda82ab72
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0ba27ce5-d963-45eb-b3b6-56afda82ab72?wsid=/subscriptions/18232d0b-4c73-4051-8694-16fb46b5a555/resourcegroups/myfreeresource/workspaces/myfreeworkspace&tid=210e9353-d35d-46b6-a109-61879c12384a


In [14]:
!pip uninstall -y ipywidgets azureml-widgets
!pip install ipywidgets==7.6.5 azureml-widgets --no-cache-dir
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

Found existing installation: ipywidgets 7.0.0
Uninstalling ipywidgets-7.0.0:
  Successfully uninstalled ipywidgets-7.0.0
Found existing installation: azureml-widgets 1.60.0
Uninstalling azureml-widgets-1.60.0:
  Successfully uninstalled azureml-widgets-1.60.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 159.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 212.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.0.8
    Uninstalling widgetsnbextension-3.0.8:
      Successfully uninstalled widgetsnbextension-3.0.8


2025-06-21 05:08:18.986634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750482500.571093    7163 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750482501.012277    7163 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750482505.205924    7163 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750482505.205982    7163 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750482505.205985    7163 computation_placer.cc:177] computation placer alr

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported


In [15]:
pipeline_run.wait_for_completion()

PipelineRunId: 0ba27ce5-d963-45eb-b3b6-56afda82ab72
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0ba27ce5-d963-45eb-b3b6-56afda82ab72?wsid=/subscriptions/18232d0b-4c73-4051-8694-16fb46b5a555/resourcegroups/myfreeresource/workspaces/myfreeworkspace&tid=210e9353-d35d-46b6-a109-61879c12384a
PipelineRun Status: Running


StepRunId: dd73d8a8-9ab8-4027-9444-95ad1f0687ed
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/dd73d8a8-9ab8-4027-9444-95ad1f0687ed?wsid=/subscriptions/18232d0b-4c73-4051-8694-16fb46b5a555/resourcegroups/myfreeresource/workspaces/myfreeworkspace&tid=210e9353-d35d-46b6-a109-61879c12384a
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)
{'runId': 'dd73d8a8-9ab8-4027-9444-95ad1f0687ed', 'target': 'mycompute', 'status': 'Completed', 'startTimeUtc': '20

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [16]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/dd73d8a8-9ab8-4027-9444-95ad1f0687ed/metrics_data, 1 files out of an estimated total of 1


In [17]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_0,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_2,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_3,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_1,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_4,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_6,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_5,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_9,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_7,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_8,...,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_11,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_16,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_12,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_17,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_19,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_21,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_15,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_13,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_20,dd73d8a8-9ab8-4027-9444-95ad1f0687ed_18
average_precision_score_micro,[0.9819952681860468],[0.8155302780083067],[0.9781010024253096],[0.9807538341541641],[0.9809765683474916],[0.9769640089166927],[0.97676686588052],[0.9803701087854335],[0.9711419140388251],[0.9505160122429386],...,[0.9801650162902739],[0.9696910741959741],[0.970778311504523],[0.9745566864433407],[0.9024364890943367],[0.9813314762735412],[0.9238344319499465],[0.8885313758169244],[0.981250391144424],[0.9809459840335076]
f1_score_weighted,[0.911845324880639],[0.7750974128113258],[0.9113535899565596],[0.9094845946984345],[0.9043652031639068],[0.9001333300643604],[0.8364342946419093],[0.9057358208500238],[0.8982724978377397],[0.8353272409172601],...,[0.9042341891559517],[0.8983599180632323],[0.8992066865342119],[0.8716648040907577],[0.850517657950007],[0.9122851253950763],[0.843353411298853],[0.8705682321298186],[0.9159685322146538],[0.9074069100714501]
recall_score_micro,[0.915113808801214],[0.7277086494688924],[0.9147192716236724],[0.9124127465857359],[0.9133535660091047],[0.9088012139605463],[0.8884370257966616],[0.9128376327769347],[0.9084066767830045],[0.8879817905918058],...,[0.9128679817905917],[0.9087101669195752],[0.908649468892261],[0.9008497723823975],[0.8220940819423369],[0.9159939301972686],[0.8125037936267072],[0.8491957511380882],[0.9176327769347496],[0.9139605462822459]
AUC_micro,[0.9811829806047238],[0.837271752620999],[0.9785452069052065],[0.9798810723932201],[0.9801680386662092],[0.976414376866591],[0.975726472030782],[0.9796448198286363],[0.9739301143729522],[0.9524390245025686],...,[0.979568647949139],[0.973173765373111],[0.9738219263564375],[0.9734070981691577],[0.8988263819969099],[0.9806646111618974],[0.9181685268294032],[0.9069164296849277],[0.9806212751651581],[0.9802487721083815]
precision_score_micro,[0.915113808801214],[0.7277086494688924],[0.9147192716236724],[0.9124127465857359],[0.9133535660091047],[0.9088012139605463],[0.8884370257966616],[0.9128376327769347],[0.9084066767830045],[0.8879817905918058],...,[0.9128679817905917],[0.9087101669195752],[0.908649468892261],[0.9008497723823975],[0.8220940819423369],[0.9159939301972686],[0.8125037936267072],[0.8491957511380882],[0.9176327769347496],[0.9139605462822459]
matthews_correlation,[0.544034395652931],[0.29648834181566863],[0.5412184239924811],[0.5330275615854683],[0.49598460594613814],[0.4759756782732981],[0.060284225829192004],[0.5073789769247323],[0.46089525324842384],[0.006870202071082593],...,[0.4970243354942615],[0.4615409972084807],[0.4664354656621093],[0.32782113800483514],[0.522457001763766],[0.54539544770832],[0.515513781190787],[0.544384689758108],[0.5706428007707647],[0.5140519046767991]
norm_macro_recall,[0.5031621321379],[0.43796978888911153],[0.49974615008877354],[0.49843683100751124],[0.4012556004206983],[0.3961310474137466],[0.004316718400393027],[0.4337708996293852],[0.367405264908207],[0.00026631158455394657],...,[0.4098655354659105],[0.3655642748654545],[0.3776193700531516],[0.16218586932407422],[0.7391399721564194],[0.4989567080055094],[0.7391063513227325],[0.7326589135025536],[0.5477948830416854],[0.44066696211833545]
accuracy,[0.915113808801214],[0.7277086494688924],[0.9147192716236724],[0.9124127465857359],[0.91335356600

### Retrieve the Best Model

In [18]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/dd73d8a8-9ab8-4027-9444-95ad1f0687ed/model_data, 1 files out of an estimated total of 1


In [19]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/irafayabdul1/code/Users/irafayabdul')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labe..._samples_split=0.15052631578947367, n_jobs=1, oob_score=True))]))], flatten_transform=False, weights=[0.07142857142857142, 0.35714285714285715, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [20]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('3', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.01, gamma=0.01, max_depth=7, max_leaves=31, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'process

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [21]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

MyFreeWorkspace
myfreeresource
eastus2
18232d0b-4c73-4051-8694-16fb46b5a555


In [22]:
# NOTE: update these to match your existing experiment name and a previous experiment
experiment_name = 'myfirstauroml'
project_folder = './pipeline-bankmarketing-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
myfirstauroml,MyFreeWorkspace,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
from azureml.pipeline.core import PipelineRun

run_id = "0ba27ce5-d963-45eb-b3b6-56afda82ab72"
pipeline_run = PipelineRun(experiment, run_id)

In [26]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bank Marketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bank Marketing Train,75038785-81b8-45e9-ad4e-8c1d030d9de5,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [27]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [28]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bankmarketing-rest-endpoint"}
                        )

In [29]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  042c63da-a0a2-494d-a826-2613235a33a9


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [30]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bankmarketing-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
